In [3]:
#tabular data imports :
import pandas as pd
import numpy as np
import env
from env import user, password, host
from pydataset import data

# visualization imports:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

# success metrics from earlier in the week: mean squared error and r^2 explained variance
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures

#stats
from scipy.stats import pearsonr, spearmanr
from scipy.stats import shapiro

import warnings
warnings.filterwarnings("ignore")
import wrangle as w
import os
directory = os.getcwd()

In [4]:
df = w.acquire_zillow()
df.head(3)

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,buildingclasstypeid,buildingclassdesc,heatingorsystemtypeid,heatingorsystemdesc,propertylandusetypeid,propertylandusedesc,storytypeid,storydesc,typeconstructiontypeid,typeconstructiondesc
0,0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,NaN,None,NaN,None,269,Planned Unit Development,NaN,None,NaN,None
1,1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,NaN,None,NaN,None,261,Single Family Residential,NaN,None,NaN,None
2,2,10843547,NaN,NaN,NaN,0.0,0.0,5.0,NaN,NaN,...,5.0,Specialized buildings that do not fit in any of the above categories,NaN,None,47,Store/Office (Mixed Use),NaN,None,NaN,None


In [5]:
w.summarize(df)

--------------------------------
--------------------------------
Information on DataFrame: 
Shape of Dataframe: (77613, 79)
--------------------------------
Basic DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77613 entries, 0 to 77612
Data columns (total 79 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            77613 non-null  int64  
 1   parcelid                      77613 non-null  int64  
 2   airconditioningtypeid         21456 non-null  float64
 3   architecturalstyletypeid      151 non-null    float64
 4   basementsqft                  45 non-null     float64
 5   bathroomcnt                   77613 non-null  float64
 6   bedroomcnt                    77613 non-null  float64
 7   buildingclasstypeid           383 non-null    float64
 8   buildingqualitytypeid         50321 non-null  float64
 9   calculatedbathnbr             74390 non-null  float64
 10